In [310]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn import neighbors

In [293]:
def draw_ellipse(canvas,a,b):
    #draw a random ellipse

    h,w = canvas.shape[0], canvas.shape[1]
    center_y = int((h-1)/2)  #np.random.randint(-h/4,h/4) + int(h/2)
    center_x = int((w-1)/2) #np.random.randint(-w/4,w/4) + int(w/2)
    axis1 = np.random.randint(a/2,a)
    axis2 = np.random.randint(b/2,b)
    angle = np.random.rand() * 90

    return cv2.ellipse(canvas,(center_x,center_y),(axis1,axis2),angle, 0,360,(1,1,1),thickness=-1)

In [ ]:
def draw_polygon(canvas,low_num_pts,high_num_pts):
    #draw a random polygon whose number of sides is between low_num and high_num

    img_height = canvas.shape[0]
    img_width = canvas.shape[1]
    num_pts = np.random.randint(low = low_num_pts,high = high_num_pts+1)

    pts = np.random.randint(low = (img_height * 0.25), high = (img_height * 0.75),size = [num_pts,2])
    pts = pts - np.array([10,10])


    binary = cv2.fillConvexPoly(np.zeros(canvas.shape),pts,(1,1,1))
    M = cv2.moments(binary)
    cX = int((img_width-1)/2 - M["m10"] / M["m00"] )
    cY = int((img_height-1)/2 -M["m01"] / M["m00"] )
    center = np.array([cX,cY])

    pts = pts + center

    recentered = cv2.fillConvexPoly(canvas,pts,(1,1,1))

    return recentered

In [294]:
def get_rand_vec(num_vecs):
    #generates a random vector
    vec = np.random.rand(num_vecs,2)
    magnitudes = np.sqrt(np.sum(vec**2,axis = 1))
    vec = vec / np.reshape(magnitudes,[num_vecs,1])

    return vec


In [295]:
def find_vec_len(img,vecs,center,max_iter):
    #extends out one at a time, check if the pixel
    #vec must have magnitude 1

    to_ret = []
    for v_num in range(vecs.shape[0]):
        for i in range(max_iter):
            pos_to_check = np.round(center + i * vecs[v_num])
            if img[int(pos_to_check[0]),int(pos_to_check[1])] < 1:
                to_ret.append(i-1)
                break

    return np.array(to_ret)

In [ ]:
def get_length_encoding(canvass,num_samples,bins):
    #get the length encoding of a binary image

    num_drawings = canvass.shape[0]
    canvas_width = canvass.shape[2]
    canvas_height= canvass.shape[1]

    center = np.array([(canvas_height-1) / 2, (canvas_width-1) / 2])
    lengthss = []

    vecs = get_rand_vec(num_samples)

    for i in range(num_drawings):
        lengths = find_vec_len(canvass[i],vecs,center,int(1.41*(canvas_width-1)/2))
        hist,bin_edges = np.histogram(lengths,bins = bins,density=True)
        lengthss.append(hist)

    lengthss = np.array(lengthss)

    return lengthss

In [ ]:
def imgs_to_adj_mat(imgs):
    #convert binary image to adjacency matrix
    num_imgs = imgs.shape[0]
    to_ret = []
    for img_idx in range(num_imgs):
        nonzero_x, nonzero_y = np.nonzero(imgs[img_idx])
        coords = np.stack([nonzero_x, nonzero_y]).T
        adj_mat = neighbors.radius_neighbors_graph(coords, np.sqrt(2))

        to_ret.append(adj_mat)

    return to_ret

In [ ]:
def adj_mat_to_lap_mat(adj_mats):
    #inputs a list of adjacency matrices, outputs a list of laplacian matrices
    num_mats = len(adj_mats)
    to_ret = []

    for mat_idx in range(num_mats):
        adj_mat = adj_mats[mat_idx]
        diag_mat = np.diag(np.sum(adj_mat,axis = 0))
        lap_mat = diag_mat - adj_mat
        #gr = nx.from_scipy_sparse_array(adj_mat)
        #lap_mat = nx.laplacian_matrix(gr).toarray()
        to_ret.append(lap_mat)
    return to_ret

In [ ]:
def lap_mat_to_eigen(lap_mats):
    #compute the eigenvalues and eigenvectors of a list of laplacian matrices
    num_mats = len(lap_mats)

    evals = []
    Us = []

    for mat_idx in range(num_mats):
        lap_mat = lap_mats[mat_idx]
        eval, U = np.linalg.eigh(lap_mat)
        evals.append(eval)
        Us.append(U)

    return evals, Us

In [ ]:
def heat_kernel_t(t, evals, U):
    #find the heat kernel at time t
    exp = np.exp(t * evals)
    lamb = np.diag(exp)
    return U.dot(lamb).dot(U.T)

In [ ]:
def heat_kernel_signature(ts, eval, U):
    #find the heat kernel signature given a bunch of ts
    toret = np.empty([ts.shape[0]])
    for i in range(ts.shape[0]):
        t = ts[i]
        kernel = heat_kernel_t(t, eval, U)
        diagonal = np.diag(kernel)
        toret[i] = np.mean(diagonal)

    return toret

In [ ]:
def batch_heat_kernel_signature(ts,evals,Us):
    #find heat kernel signature given a bunch of ts of a bunch of matrices
    num_mats = len(evals)
    to_ret = np.empty((num_mats,ts.shape[0]))

    for mat_idx in range(num_mats):
        hks = heat_kernel_signature(ts,evals[mat_idx],Us[mat_idx])
        to_ret[mat_idx] = hks

    return to_ret